In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import time

from symbol import Symbol
from helpers import rename_columns, SafeHDFStore, clean_file

import warnings
warnings.filterwarnings('ignore')

In [69]:

jss = json.loads(open('edel.har', 'r').read())

In [1]:
url =  'https://ewmw.edelweiss.in/api/Market/Process/GetEquitiesSymbolDetails'
headers = {
    'Host': 'ewmw.edelweiss.in',
    'Connection': 'keep-alive',
    'Origin': 'https://www.edelweiss.in',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Access-Control-Request-Headers': 'content-type',
    'Accept': '*/*',
    'Referer': 'https://www.edelweiss.in/quotes/equity/Andhra-Cements-Ltd-19?q=andhra%20cements%20ltd',
    'Accept-Encoding': 'gzip, deflate, sdch, br',
    'Accept-Language': 'en-IN,en;q=0.8',    
    "Accept" : "application/json, text/plain, */*",
    "Content-Type" : "application/json; charset=UTF-8",
    "Content-Length" : "20"
}
# Adding empty header as parameters are being sent in payload
payload = {"syLst":["ANDCEM"]}
r = requests.post(url, data=json.dumps(payload), headers=headers, verify=False)
rx = requests.get('https://z.webengage.co/gz.js')
px = requests.get('https://c.webengage.com/upf.js?&lp=https://www.edelweiss.in/quotes/equity/Infosys-Ltd-2806?q=infy&type=relevance&rf=&geo=y&jsonp=_we_jsonp_global_cb_1485005650151')

NameError: name 'requests' is not defined

In [188]:
(json.loads(str(r.text)))
r.url
rx
px.text

'_we_jsonp_global_cb_1485005650151({"geo":{"clientIp":"112.133.208.178","serverTimeStamp":1485006016450,"country":"India","region":null,"city":null,"geoplugin_countryName":"India","geoplugin_region":null,"geoplugin_city":null},"acquisitionData":{"referrer_type":"direct","landing_page":"https://www.edelweiss.in/quotes/equity/Infosys-Ltd-2806?q=infy"},"ua":{"browser":{},"os":{},"device":"desktop"},"ts":1485006016451})'

In [104]:
liss = jss['log']['entries']
for ls in liss:
    if ls['request']['url'] == 'https://ewmw.edelweiss.in/api/Market/Process/GetEquitiesSymbolDetails':
        mys = ls['request']
mys

{'bodySize': 0,
 'cookies': [],
 'headers': [{'name': 'Accept', 'value': 'application/json, text/plain, */*'},
  {'name': 'Referer',
   'value': 'https://www.edelweiss.in/quotes/equity/Andhra-Cements-Ltd-19?q=andhra%20cements%20ltd'},
  {'name': 'Origin', 'value': 'https://www.edelweiss.in'},
  {'name': 'User-Agent',
   'value': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/538.1 (KHTML, like Gecko) splash Safari/538.1'},
  {'name': 'Content-Type', 'value': 'application/json;charset=UTF-8'},
  {'name': 'Content-Length', 'value': '20'}],
 'headersSize': 342,
 'httpVersion': 'HTTP/1.1',
 'method': 'POST',
 'queryString': [],
 'url': 'https://ewmw.edelweiss.in/api/Market/Process/GetEquitiesSymbolDetails'}

In [111]:
listss= []
for head in mys['headers']:
     listss.append('"{0}" : "{1}"'.format(head.get('name'), head.get('value')))
import pprint
pprint.pprint('{'+', '.join(listss)+'}')

('{"Accept" : "application/json, text/plain, */*", "Referer" : '
 '"https://www.edelweiss.in/quotes/equity/Andhra-Cements-Ltd-19?q=andhra%20cements%20ltd", '
 '"Origin" : "https://www.edelweiss.in", "User-Agent" : "Mozilla/5.0 (X11; '
 'Linux x86_64) AppleWebKit/538.1 (KHTML, like Gecko) splash Safari/538.1", '
 '"Content-Type" : "application/json;charset=UTF-8", "Content-Length" : "20"}')


In [106]:
json.dumps('{'+', '.join(listss)+'}')

'"{\\"Accept\\" : \\"application/json, text/plain, */*\\", \\"Referer\\" : \\"https://www.edelweiss.in/quotes/equity/Andhra-Cements-Ltd-19?q=andhra%20cements%20ltd\\", \\"Origin\\" : \\"https://www.edelweiss.in\\", \\"User-Agent\\" : \\"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/538.1 (KHTML, like Gecko) splash Safari/538.1\\", \\"Content-Type\\" : \\"application/json;charset=UTF-8\\", \\"Content-Length\\" : \\"20\\"}"'

In [190]:
psd = requests.get('https://ewstream.edelweiss.in/signalr/start?transport=webSockets&clientProtocol=1.5&connectionToken=AQAAANCMnd8BFdERjHoAwE%2FCl%2BsBAAAAkehhiqPHgE6YOI6ZrMjuFAAAAAACAAAAAAAQZgAAAAEAACAAAAChZ39t6cbr7goyVqlnKxuAuP6tw%2BrH7pX%2BTK8TleDxVQAAAAAOgAAAAAIAACAAAADLyZF%2BQo0KHhoMFw6LIAm3YXivnGROiUHDS6010iFo9zAAAABW7PzYR1BhaQPzbMpSWH535F%2BremEjXjWiMgFs6AsuM3cIMtxUEr5ZlSoDH4zu%2BHlAAAAA8FKuyICT5OJahRdPQz3FsH2rgZ38%2F3sMFaUc7eLkDJqrb220ptDUH%2FK3pwOYkUIC4EYsvNXkhAkicS8DLt8pnQ%3D%3D&connectionData=%5B%7B%22name%22%3A%22stockfeedhub%22%7D%5D&_=1485003929994')

In [192]:
psd.text

'{ "Response": "started" }'